## 1️⃣ Importar Librerías

In [1]:
import sys
import time
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Agregar path del proyecto
sys.path.insert(0, str(Path.cwd()))

# Librerías
import pandas as pd

# Nuestro modelo
from ml_model.model import VulnerabilityPredictor
from scripts.code_analyzer import CodeAnalyzer

print("✅ Librerías importadas")

✅ Librerías importadas


## 2️⃣ Cargar Dataset

In [2]:
print("📥 Cargando datasets de Python y JavaScript...\n")
start_time = time.time()

# Cargar ambos datasets
df_python = pd.read_csv('Dataset/data_Python.csv')
df_js = pd.read_csv('Dataset/data_JavaScript.csv')

print(f"✅ Python: {len(df_python):,} muestras")
print(f"✅ JavaScript: {len(df_js):,} muestras")

# Combinar datasets
df = pd.concat([df_python, df_js], ignore_index=True)

print(f"\n✅ Dataset combinado: {len(df):,} muestras totales")
print(f"   Vulnerables: {df['is_vulnerable'].sum():,}")
print(f"   Seguras: {(len(df)-df['is_vulnerable'].sum()):,}")

📥 Cargando datasets de Python y JavaScript...

✅ Python: 2,316 muestras
✅ JavaScript: 39,978 muestras

✅ Dataset combinado: 42,294 muestras totales
   Vulnerables: 42,294
   Seguras: 0
✅ Python: 2,316 muestras
✅ JavaScript: 39,978 muestras

✅ Dataset combinado: 42,294 muestras totales
   Vulnerables: 42,294
   Seguras: 0


## 3️⃣ Extraer Características

In [3]:
print("🔍 Analizando código vulnerable y seguro (patch)...")
print(f"⏳ Esto tomará ~8-10 minutos para {len(df)*2:,} muestras (Python + JavaScript)...\n")

analyzer = CodeAnalyzer()
features_list = []
labels = []
failed = 0

# Procesar código vulnerable (vul_code)
print("📌 Procesando código VULNERABLE...")
for idx, row in df.iterrows():
    if idx % 500 == 0:
        print(f"   Procesado: {idx}/{len(df)} ({idx/len(df)*100:.1f}%)")
    
    try:
        temp_file = "temp_analysis.py"
        with open(temp_file, 'w', encoding='utf-8') as f:
            f.write(row['vul_code'])
        
        result = analyzer.analyze_file(temp_file)
        features_list.append(result['features'])
        labels.append(1)  # Vulnerable
        
        Path(temp_file).unlink(missing_ok=True)
    except:
        failed += 1

# Procesar código seguro (patch)
print(f"\n📌 Procesando código SEGURO (patch)...")
for idx, row in df.iterrows():
    if idx % 500 == 0:
        print(f"   Procesado: {idx}/{len(df)} ({idx/len(df)*100:.1f}%)")
    
    try:
        temp_file = "temp_analysis.py"
        with open(temp_file, 'w', encoding='utf-8') as f:
            f.write(row['patch'])
        
        result = analyzer.analyze_file(temp_file)
        features_list.append(result['features'])
        labels.append(0)  # Seguro
        
        Path(temp_file).unlink(missing_ok=True)
    except:
        failed += 1

X = pd.DataFrame(features_list)
y = pd.Series(labels)

print(f"\n✅ Análisis completo: {len(features_list):,} válidas, {failed} fallidas")
print(f"   Vulnerables: {y.sum():.0f} ({y.sum()/len(y)*100:.1f}%)")
print(f"   Seguras: {(len(y)-y.sum()):.0f} ({(len(y)-y.sum())/len(y)*100:.1f}%)")

🔍 Analizando código vulnerable y seguro (patch)...
⏳ Esto tomará ~8-10 minutos para 84,588 muestras (Python + JavaScript)...

📌 Procesando código VULNERABLE...
   Procesado: 0/42294 (0.0%)
   Procesado: 500/42294 (1.2%)
   Procesado: 500/42294 (1.2%)
   Procesado: 1000/42294 (2.4%)
   Procesado: 1000/42294 (2.4%)
   Procesado: 1500/42294 (3.5%)
   Procesado: 1500/42294 (3.5%)
   Procesado: 2000/42294 (4.7%)
   Procesado: 2000/42294 (4.7%)
   Procesado: 2500/42294 (5.9%)
   Procesado: 2500/42294 (5.9%)
   Procesado: 3000/42294 (7.1%)
   Procesado: 3000/42294 (7.1%)
   Procesado: 3500/42294 (8.3%)
   Procesado: 3500/42294 (8.3%)
   Procesado: 4000/42294 (9.5%)
   Procesado: 4000/42294 (9.5%)
   Procesado: 4500/42294 (10.6%)
   Procesado: 4500/42294 (10.6%)
   Procesado: 5000/42294 (11.8%)
   Procesado: 5000/42294 (11.8%)
   Procesado: 5500/42294 (13.0%)
   Procesado: 5500/42294 (13.0%)
   Procesado: 6000/42294 (14.2%)
   Procesado: 6000/42294 (14.2%)
   Procesado: 6500/42294 (15.4%)
   P

## 4️⃣ Entrenar Modelo

In [4]:
print("🧠 Entrenando modelo Random Forest...\n")

predictor = VulnerabilityPredictor()
metrics = predictor.train(X, y)

print(f"\n✅ Modelo entrenado")
print(f"   Accuracy: {metrics['accuracy']:.2%}")
print(f"   ROC-AUC: {metrics['roc_auc']:.4f}")

🧠 Entrenando modelo Random Forest...

Entrenando modelo Random Forest...

Accuracy: 1.0000
ROC-AUC: 1.0000
Cross-validation: 0.9456 (+/- 0.0981)

✅ Modelo entrenado
   Accuracy: 100.00%
   ROC-AUC: 1.0000

Accuracy: 1.0000
ROC-AUC: 1.0000
Cross-validation: 0.9456 (+/- 0.0981)

✅ Modelo entrenado
   Accuracy: 100.00%
   ROC-AUC: 1.0000


## 5️⃣ Guardar Modelo

In [5]:
model_path = "ml_model/vulnerability_detector.pkl"

print("💾 Guardando modelo...")
predictor.save_model(model_path)

model_size = Path(model_path).stat().st_size / (1024 * 1024)
total_time = time.time() - start_time

print(f"\n✅ COMPLETADO")
print(f"   Archivo: {model_path}")
print(f"   Tamaño: {model_size:.2f} MB")
print(f"   Tiempo total: {total_time/60:.2f} minutos")
print(f"\n🚀 USAR EL MODELO:")
print(f"   python scripts/vulnerability_scanner.py <archivo.py>")
print(f"   python scripts/report_generator.py reports/scan_results.json")

💾 Guardando modelo...
Modelo guardado en: ml_model/vulnerability_detector.pkl

✅ COMPLETADO
   Archivo: ml_model/vulnerability_detector.pkl
   Tamaño: 0.19 MB
   Tiempo total: 15.22 minutos

🚀 USAR EL MODELO:
   python scripts/vulnerability_scanner.py <archivo.py>
   python scripts/report_generator.py reports/scan_results.json
